In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Python的Vertex AI SDK：为在线预测训练AutoML文本实体提取模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_text_entity_extraction_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_text_entity_extraction_online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/sdk_automl_text_entity_extraction_online.ipynb" target='_blank'>     
    <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程展示了如何使用Vertex AI SDK for Python创建文本实体提取模型，并使用Google Cloud的AutoML模型进行在线预测。

了解更多关于[文本数据的实体提取](https://cloud.google.com/vertex-ai/docs/training-overview#entity_extraction_for_text)。

### 目标

在本教程中，您将学习如何使用Vertex AI SDK从Python脚本创建并部署一个AutoML文本实体提取模型，以进行在线预测。您也可以选择使用`gcloud`命令行工具或在Cloud Console上在线创建和部署模型。

本教程使用以下谷歌云机器学习服务：

- `AutoML Training`
- `Vertex AI Datasets`

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到一个服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

### 数据集

本教程使用的数据集是来自[国家生物技术信息中心](https://www.ncbi.nlm.nih.gov/)的[NCBI疾病研究摘要数据集](https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/)。您将在本教程中使用的数据集版本存储在一个公共云存储桶中。

### 费用

本教程使用 Google Cloud 的收费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
定价](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成费用估算。

安装

安装最新版本的Python版Vertex AI SDK。

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage

只有Colab: 取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[找到项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的`REGION`变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 因为您已经经过验证，所以不需要做任何操作。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples上为您的服务帐户授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时，运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import os

import google.cloud.aiplatform as aiplatform

初始化用于Python的Vertex SDK

为您的项目和对应的存储桶初始化用于Python的Vertex SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# 教程

现在你已经准备好开始创建自己的AutoML文本实体提取模型。

请将云存储培训数据的位置设置为变量'IMPORT_FILE'，该变量应指向Cloud Storage中JSONL索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/language/ucaip_ten_dataset.jsonl"

快速查看您的数据

本教程使用存储在公共云存储桶中的NCBI生物医学数据集的一个版本，使用一个JSONL索引文件。

首先，快速查看数据。通过计算JSONL索引文件中对象的数量（`wc -l`），然后查看前几行。

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用`TextDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源。
- `import_schema_uri`：数据项的数据标记模式。

这个操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.TextDataset.create(
    display_name="NCBI Biomedical",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.extraction,
)

print(dataset.resource_name)

创建和运行训练管道

要训练一个AutoML模型，需要执行两个步骤：1）创建一个训练管道，2）运行该管道。

创建训练管道

使用`AutoMLTextTrainingJob`类创建一个AutoML训练管道，包括以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：要为模型训练的任务类型。
  - `classification`：文本分类模型。
  - `sentiment`：文本情感分析模型。
  - `extraction`：文本实体提取模型。
- `multi_label`：如果是分类任务，则指定是单标签(False)还是多标签(True)。
- `sentiment_max`：如果是情感分析任务，则指定最大情感值。

In [ ]:
job = aiplatform.AutoMLTextTrainingJob(
    display_name="biomedical", prediction_type="extraction"
)

print(job)

#### 运行训练流水线

接下来，您可以通过调用`run`方法开始训练作业，使用以下参数：

- `dataset`：要训练模型的`Dataset`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。

完成`run`方法后，将返回`Model`资源。

训练流水线的执行时间最多需要4小时。

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name="biomedical",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

## 查看模型评估分数
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取新模型的引用。与数据集一样，您可以使用在部署模型时创建的模型变量的引用，或者列出您项目中的所有模型。

In [ ]:
# Get model resource ID
models = aiplatform.Model.list(filter="display_name=biomedical")

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aiplatform.gapic.ModelServiceClient(
    client_options=client_options
)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

## 部署模型

接下来，部署您的模型用于在线预测。要部署模型，您需要调用`deploy`方法。

In [ ]:
endpoint = model.deploy()

发送在线预测请求

向您部署的模型发送在线预测。

### 创建测试项目

您将使用合成数据作为测试数据项目。不要担心我们使用的是合成数据 - 我们只是想展示如何进行预测。

In [ ]:
test_item = 'Molecular basis of hexosaminidase A deficiency and pseudodeficiency in the Berks County Pennsylvania Dutch.\tFollowing the birth of two infants with Tay-Sachs disease ( TSD ) , a non-Jewish , Pennsylvania Dutch kindred was screened for TSD carriers using the biochemical assay . A high frequency of individuals who appeared to be TSD heterozygotes was detected ( Kelly et al . , 1975 ) . Clinical and biochemical evidence suggested that the increased carrier frequency was due to at least two altered alleles for the hexosaminidase A alpha-subunit . We now report two mutant alleles in this Pennsylvania Dutch kindred , and one polymorphism . One allele , reported originally in a French TSD patient ( Akli et al . , 1991 ) , is a GT-- > AT transition at the donor splice-site of intron 9 . The second , a C-- > T transition at nucleotide 739 ( Arg247Trp ) , has been shown by Triggs-Raine et al . ( 1992 ) to be a clinically benign " pseudodeficient " allele associated with reduced enzyme activity against artificial substrate . Finally , a polymorphism [ G-- > A ( 759 ) ] , which leaves valine at codon 253 unchanged , is described'

### 进行预测

现在，您的`Model`资源已部署到一个`Endpoint`资源上，您可以通过向该`Endpoint`资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为：

     { 'content': text_string }

由于predict（）方法可以接受多个条目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

predict（）调用的响应是一个带有以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `displayNames`：每个实体的类名。
- `confidences`：每个实体的预测置信度，介于0和1之间。
- `textSegmentStartOffsets`：在文本中指向实体开始的字符偏移量。
- `textSegmentEndOffsets`：在文本中指向实体结尾的字符偏移量。
- `deployed_model_id`：执行预测的已部署`Model`资源的Vertex AI标识符。

In [ ]:
instances_list = [{"content": test_item}]

prediction = endpoint.predict(instances_list)
print(prediction)

## 下线模型

当您完成预测后，您可以从`Endpoint`资源中下线模型。这会取消所有计算资源，并结束已部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的个别资源：

- 数据集
- 流水线
- 模型
- 端点
- AutoML训练作业
- 批处理作业
- 自定义作业
- 超参数调整作业
- Cloud Storage存储桶

In [ ]:
# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the AutoML or Pipeline training job
job.delete()

if os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI